In [ ]:
!pip install -U gspread
!pip install -U google-api-python-client oauth2client

In [4]:
import locale
import gdown

import pandas as pd


from docxtpl import DocxTemplate
from datetime import datetime
from googletrans import Translator

import smtplib
import ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

translator = Translator()

ModuleNotFoundError: No module named 'termios'

In [ ]:
def getEMails():
  gs = gc.open('DBS REPORTE DE VENTAS').worksheet('eMails')
  data = gs.get_all_values()
  headers = data.pop(0)
  df = pd.DataFrame(data, columns=headers)
  return df

def getDataFrame(fuente):
  gs = gc.open('DBS REPORTE DE VENTAS').worksheet(fuente)
  data = gs.get_all_values()
  headers = data.pop(0)
  df = pd.DataFrame(data, columns=headers)
  df.columns = df.iloc[0]
  df = df[1:]
  df = df.iloc[:,1:-7]
  df.drop(df[df['NOMBRE ASESOR(A)'] == ''].index, inplace = True)
  df = df.loc[:, ['PERIODO', # No entra a la tabla
              'NOMBRE ASESOR(A)', # No entra a la tabla
              'UNIDAD DE NEGOCIO CORRESPONDIENTE', # Entra por separado
              'NOMBRE PROGRAMA',
              'NOMBRE INSCRITO',
              'FECHA DE INSCRIPCION PORTAL',
              'PROMOCIONES',
              'MODALIDAD DE PAGOS',
              'CANTIDAD DE CUOTAS',
              'COLEGIATURA A RECOLECTAR (SOLO MONTO COLEGIATURA)',
              'M FECHA RECIBO',
              'M N° RECIBO',
              'M MONTO RECIBO',
              'C FECHA RECIBO',
              'C MONTO RECIBO',
              'C N° RECIBO',
              'CORRESP A META DE SUELDO?',
              'COMISION SEGUN TABLA',
              'COMISION PLAN CONTADO',
              'TOTAL COMISION',
              # datos para la tabla de documentos entregados
              'CI',
              'TITULO EN PROV NAL',
              'DIPLOMA ACADEMICO',
              'CERTIFICADO DE NACIMIENTO',
              'CARTA DE INSCRIPCIÓN',
              'FORMULARIO DE INSCRIPCIÓN',
              'CURRICULUM DOCUMENTADO',
              'ESTADO DE INSCRIPCIÓN',
            ]
      ]
  return df

def getReport(df, mes, ano, asesor):
  reporte = df.loc[
      (df['PERIODO'] == f'01/{mes}/{ano}')
      & (df['NOMBRE ASESOR(A)'].str.contains(asesor))
  ]
  x, y = reporte.shape
  reporte = reporte.sort_values('UNIDAD DE NEGOCIO CORRESPONDIENTE', ascending=True)

  reporte.reset_index(drop=True, inplace=True)
  reporte.index += 1
  reporte['N°'] = reporte.index
  col_indices = reporte['N°']
  reporte = reporte.drop(columns=['N°'])
  reporte.insert(0, 'N°', col_indices)
  reporte.reset_index(drop=True)

  eliminar = df.columns[df.columns.get_loc('FECHA DE INSCRIPCION PORTAL'):df.columns.get_loc('TOTAL COMISION') + 1]
  documentos = reporte.drop(columns=eliminar)
  eliminar = df.columns[df.columns.get_loc('CI'):df.columns.get_loc('CURRICULUM DOCUMENTADO') + 1]
  reporte = reporte.drop(columns=eliminar)
  if x > 0:
    asesor = reporte['NOMBRE ASESOR(A)'][1].title()
    periodo = datetime.strptime(reporte['PERIODO'][1], "%d/%m/%Y").strftime("mes de %B del %Y")
    periodo = translator.translate(periodo, src='en', dest='es').text
  else:
    asesor = _asesor
    periodo = f'01/{mes}/{ano}'

  reporte = reporte.drop(columns=['NOMBRE ASESOR(A)', 'PERIODO', 'ESTADO DE INSCRIPCIÓN'])
  documentos = documentos.drop(columns=['NOMBRE ASESOR(A)', 'PERIODO'])

  return (reporte, documentos, asesor, periodo)

def getBusinesses(reporte):
  unidades_negocio = reporte['UNIDAD DE NEGOCIO CORRESPONDIENTE'].unique().tolist()
  aux = reporte['UNIDAD DE NEGOCIO CORRESPONDIENTE'].tolist()
  indexes = [aux.index(unidad) for unidad in unidades_negocio]
  indexes.append(len(aux))
  indexes = [(indexes[i], indexes[i + 1]) for i in range(0, len(indexes) - 1)]
  return indexes, unidades_negocio

def getHeaders(reporte, documentos):
  headers_r = reporte.columns.tolist()
  headers_r = headers_r[2:]
  headers_d = documentos.columns.tolist()
  headers_d = headers_d[3:]
  return (headers_r, headers_d)

def getFormatedReport(headers, reporte, documentos, indexes, unidades_negocio):
  reporte_dict = reporte.to_dict(orient='index')
  reporte_list = []
  for _, elemento in reporte_dict.items():
    nuevo = {'N°': elemento['N°'], 'Unidad de Negocio': elemento['UNIDAD DE NEGOCIO CORRESPONDIENTE']}
    lista = [x for _, x in elemento.items()]
    lista = lista[2:]
    nuevo['lista'] = lista
    reporte_list.append(nuevo)

  documentos_dict = documentos.to_dict(orient='index')
  documentos_list = []
  for _, item in documentos_dict.items():
    nuevo = {'N°': item['N°']}
    lista = [x for _, x in item.items()]
    lista = lista[3:]
    nuevo['lista'] = lista
    documentos_list.append(nuevo)

  reporte_tup = []
  for x, y in indexes:

    sublista = reporte_list[x:y]
    subdocumentos= documentos_list[x:y]
    un_negocio = unidades_negocio.pop(0)

    for fila in sublista:
      if fila['Unidad de Negocio'] != un_negocio:
        raise ValueError('Disparidad de unidades de negocio')
      del fila['Unidad de Negocio']

    reporte_tup.append((headers, un_negocio, [(item['N°'], item['lista']) for item in sublista], [(item['N°'], item['lista']) for item in subdocumentos]))

  return reporte_tup

def printInforme(datos, asesor):
  folder = 'reportes'
  template = DocxTemplate('PlantillaInformeMKT.docx')
  template.render(datos)
  template.save(f'{folder}/ReporteMKT({asesor}).docx')
  #files.download(f'{folder}/ReporteMKT({asesor}).docx')
  print(f'Informe generado de: {asesor}')

def enviarInforme(asesor, to_addrs):
  from_addr = 'auto.dbs@dbussinesschool.com'
  subject = f'Informe {periodo}'
  content = f'Hola {asesor} le enviamos su informe como archivo adjunto'

  msg = MIMEMultipart()
  msg['From'] = from_addr
  msg['To'] = to_addrs[0]
  msg['Subject'] = subject
  body = MIMEText(content, 'plain')
  msg.attach(body)

  archivo = f'ReporteMKT({asesor}).docx'
  path = f'reportes/{archivo}'

  pdf = MIMEApplication(open(path, 'rb').read())
  pdf.add_header('Content-Disposition', 'attachment', filename=archivo)
  msg.attach(pdf)

  server = smtplib.SMTP('smtp.gmail.com', 25)
  server.connect('smtp.gmail.com', 587)
  server.verify(from_addr)
  server.ehlo()
  server.starttls()
  server.ehlo()
  server.login(from_addr, 'Auto,dbs12\'')

  #server.sendmail(from_addr, to_addrs, msg.as_string())
  server.quit()
  print(f'Informe enviado a: {to_addrs[0]}')

In [1]:
gdown.download('https://docs.google.com/document/d/1LvHDadpqLlR6qEpbq1eu7Srnye0XwS8E/edit?usp=sharing&ouid=117379727352288078520&rtpof=true&sd=true',
               'PlantillaReporteMKT',quiet=False)
files.download('PlantillaReporteMKT.docx')
!mkdir reportes
!ls

NameError: name 'gdown' is not defined

In [ ]:
df_ventas = getDataFrame('REGISTRO')
df_resagados = getDataFrame('RESAGADOS')

In [ ]:
_ano = '2023'
_mes = '10'
_asesor = 'Fabiola'.upper()

reporte, documentos, asesor, periodo = getReport(df_ventas, _mes, _ano, _asesor)
resagados, documentosr, asesorr, periodor = getReport(df_resagados, _mes, _ano, _asesor)

rep_i, rep_un = getBusinesses(reporte)
res_i, res_un = getBusinesses(resagados)

rep_headers = getHeaders(reporte, documentos)
res_headers = getHeaders(resagados, documentosr)

rep = getFormatedReport(rep_headers, reporte, documentos, rep_i, rep_un)
res = getFormatedReport(res_headers, resagados, documentosr, res_i, res_un)
datos = {
    'asesor': asesor.title(),
    'periodo': periodo,
    'fecha': translator.translate(datetime.now().strftime("%d de %B del %Y"), src='en', dest='es').text,
    'observaciones': 'Reporte generado automáticamente: (agregar su interpretación personal aquí)',
    'lista': rep,
    'resagados': res
}
!rm -rf reportes/*
printInforme(datos, datos['asesor'])

addrs = pd.DataFrame(getEMails())
to_addrs = addrs.loc[addrs['Nombre'].str.lower() == asesor.lower()]['eMail'].tolist()
enviarInforme(datos['asesor'], to_addrs)

In [ ]:
asesores = getEMails().values.tolist()
_ano = '2023'
_mes = '10'

!rm -rf reportes/*

for _asesor, eMail in asesores:
  reporte, documentos, asesor, periodo = getReport(df_ventas, _mes, _ano, _asesor)
  resagados, documentosr, asesorr, periodor = getReport(df_resagados, _mes, _ano, _asesor)

  x, y = reporte.shape
  i, j = resagados.shape
  if x <= 0 and i <= 0:
    print(f'Informe vacío de: {asesor.title()}\nInforme no enviado a: {eMail}')
    continue

  rep_i, rep_un = getBusinesses(reporte)
  res_i, res_un = getBusinesses(resagados)

  rep_headers = getHeaders(reporte, documentos)
  res_headers = getHeaders(resagados, documentosr)

  rep = getFormatedReport(rep_headers, reporte, documentos, rep_i, rep_un)
  res = getFormatedReport(res_headers, resagados, documentosr, res_i, res_un)

  datos = {
    'asesor': asesor.title(),
    'periodo': periodo,
    'fecha': translator.translate(datetime.now().strftime("%d de %B del %Y"), src='en', dest='es').text,
    'observaciones': 'Reporte generado automáticamente: (agregar su interpretación personal aquí)',
    'lista': rep,
    'resagados': res
  }

  printInforme(datos, datos['asesor'])
  enviarInforme(datos['asesor'], [eMail])